In [1]:
#To get whole monitor screen
from PIL import ImageGrab
from functools import partial
ImageGrab.grab = partial(ImageGrab.grab,all_screens = True)

In [2]:
import numpy as np
import pyautogui as pg
import cv2

In [4]:
class liner:
    def __init__(self):
        self.nowX,self.nowY = 0,0
        self.endX,self.endY = 0,0
        self.dest_angle = 0
        self.move_angle = 0
        self.map  = None

        self.threshold = 200
        self.stepsize = 20
        self.anglerange = np.pi/3
    
    def calc_angle(self,x1,y1,x2,y2):
        try:
            cosV = (x2-x1)/(((x2-x1)**2+(y2-y1)**2)**0.5)
            angle = 2*((y2>=y1)-0.5)*np.arccos(cosV)
        except ZeroDivisionError:
            angle = 0

        return angle
    
    def calc_dest_angle(self):
        return self.calc_angle(self.nowX,self.nowY,self.endX,self.endY)

    def get_map(self):
        ss = pg.screenshot()
        self.map = np.array(ss.convert('L'))
        #get_map_image
    
    def get_brightness(self,x,y):
        return self.map[y][x]



class tracker(liner):
    def __init__(self,start,end,map):
        super().__init__()
        self.nowX,self.nowY = start
        self.endX,self.endY = end
        self.map = map
        self.dest_angle = self.calc_angle(self.nowX,self.nowY,self.endX,self.endY)
        self.loclist = []
        
    def check_next(self, angle):
        nextX = int(self.nowX+self.stepsize*np.cos(angle))
        nextY = int(self.nowY+self.stepsize*np.sin(angle))
        is_bright = self.get_brightness(nextX,nextY)>=self.threshold
        
        return nextX,nextY, is_bright
    
    def find_next(self,rng = 5):
        self.dest_angle = self.calc_dest_angle()

        for i in range(rng):
            angle = self.dest_angle+i*self.anglerange/rng
            nextX, nextY, is_bright = self.check_next(angle)
            if is_bright:
                self.nowX,self.nowY = nextX,nextY
                self.loclist.append((nextX,nextY))
                return True

            angle = self.dest_angle-i*self.anglerange/rng
            nextX, nextY, is_bright = self.check_next(angle)
            if is_bright:
                self.nowX,self.nowY = nextX,nextY
                self.loclist.append((nextX,nextY))
                return True

        return False

    def line_make(self):
        max_cal = 1000
        cal = 0
        while ((self.nowX-self.endX)**2+(self.nowY-self.endY)**2)**0.5>self.stepsize or cal<max_cal:
            self.loclist.append((self.nowX,self.nowY))

            f = self.find_next()
            if f ==False:
                return self.loclist
            cal += 1

        self.loclist.append((self.endX,self.endY))
        return self.loclist

    def mouse_follow(self):
        self.line_make()
        
        pg.mouseDown(self.loclist[0])
        for loc in self.loclist[1:]:
            pg.moveTo(loc,duration=0.1,_pause=False)

        pg.mouseUp(self.loclist[0])
